# Flask + WhyLabs

This example shows how to deploy an app that integrates whylabs logging.

In [ ]:
!pip install pandas joblib scikit-learn

In [ ]:
import os
import json
import random
import datetime
import numpy as np
import time
import urllib.request as urllib
import requests
import pandas as pd
from joblib import dump
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

## Table of Contents
- [1 - Train Model](#1)
- [2 - Build a docker image with app](#2)
- [3 - Run the container to serve the API](#3)
- [4 - Test endpoint](#4)
- [5 - Check WhyLabs Platform](#5)

<a name='1'></a>
## 1 - Train Model

Download Iris Species dataset:

In [ ]:
# Download Iris dataset and save it as csv
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
raw_data = urllib.urlopen(url)
try:
    os.mkdir("dataset/")
except Exception as e:
    print(" 'dataset' directory already existed. Moving forward")
# Save data as csv
with open('dataset/Iris.csv', 'wb') as file:
    file.write(raw_data.read())

Split data set into train and test sets

In [ ]:
data = pd.read_csv('dataset/Iris.csv', header=None)

In [ ]:
# Separating the independent variables from dependent variables
X = data.iloc[:, 0:4].values
y = data.iloc[:, -1].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

Train the SVM classifier

In [ ]:
# Train a classifier
print("Train started.")
model = SVC()
model.fit(x_train, y_train)
print("Train finished.")
# Save the model
dump(model, 'model.joblib')
print("Model saved as model.joblib")

<a name='2'></a>
## 2 - Build a docker image with app

In [ ]:
!docker build --build-arg PYTHON_VERSION=3.7 -t whylabs-flask .

<a name='3'></a>
## 3 - Run the container to serve the API

Open a terminal and execute the following command:

```bash
docker run --rm -p 5000:5000 whylabs-flask
```

<a name='4'></a>
## 4 - Test endpoint

- Go to http://0.0.0.0:5000/apidocs/
- Open /predict endpoint green tab.
- Click Try it out.
- Click Execute green button.
- Check the response and code, if 200, the API is working!

The following functions aim to modify the variables distribution in order to test whylabs.

In [ ]:
def modify_random_column_values(data, value: float = np.random.uniform(low=0.0, high=10.0)) -> None:
    random_column = None
    data_mod = data.copy(deep=True)
    try:
        number_of_columns = len(data_mod.columns) - 2 # Index and label eliminated
        random_column = data_mod.columns[np.random.randint(number_of_columns) + 1]
        data_mod[random_column] = value
    except Exception as ex:
        raise f"Error adding fix value in random column: {str(random_column)}"
    return data_mod
        
        
def add_random_column_outliers(data, number_outliers: int = 10) -> None:
    random_column = None
    data_mod = data.copy(deep=True)
    try:
        number_of_columns = len(data_mod.columns) - 2  # Index and label eliminated
        number_of_rows = data_mod.shape[0]
        random_column = data_mod.columns[np.random.randint(number_of_columns) + 1]
        for i in range(number_outliers):
            random_row = np.random.randint(0, number_of_rows)
            data_mod.loc[random_row, random_column] = round(np.random.uniform(low=20.0, high=50.0), 2)
    except Exception as ex:
        raise f"Error adding outliers in random column: {random_column}"
    return data_mod

Once it's working, you can try to send continous requests to the endpoint:

In [ ]:
labels = ["sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm"]

In [ ]:
# modify a variable distribution
data_mod = add_random_column_outliers(data, 30)
print("Dataset distribution modified!")

In [ ]:
url = "http://0.0.0.0:5000/api/v1"

In [ ]:
healthy = requests.get(f"{url}/health")
if healthy.ok:
    for k in range(data_mod.shape[0]):
        # Build a payload with random values
        payload = dict(zip(labels, data_mod.iloc[:, 0:4].values[k]))
        print(payload)
        response = requests.post(f"{url}/predict", json=payload)
        if response.ok:
            print(response.json())
            time.sleep(5)

<a name='5'></a>
## 5 - Check WhyLabs Platform

- Go to https://hub.whylabsapp.com/models

![WhyLabs Models page](assets/1.png)

- Select your model (by default is model-1)

![WhyLabs model-1 page](assets/2.png)

- Select the feature of interest and explore

![WhyLabs Feature page](assets/3.png)